## Demonstrate **hklpy2**'s API

A working notebook as the package is being developed.
***Anything*** could change.

In [1]:
import datetime
import hklpy2
import math

print(f"{datetime.datetime.now()}")
print(f"{hklpy2.__version__=}")
print(f"{hklpy2.solvers()=}")

2024-05-23 16:10:24.264484
hklpy2.__version__='0.0.13.dev0+gecef31d.d20240523'
hklpy2.solvers()={'hkl_soleil': 'hklpy2.backends.hkl_soleil:HklSolver', 'no_op': 'hklpy2.backends.no_op:NoOpSolver', 'th_tth': 'hklpy2.backends.th_tth_q:ThTthSolver'}


In [2]:
from ophyd import Component as Cpt
from ophyd import Kind
from ophyd import PseudoSingle
from ophyd import SoftPositioner

NORMAL_HINTED = Kind.hinted | Kind.normal

class Fourc(hklpy2.DiffractometerBase):
    """Test case."""

    h = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741
    k = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741
    l = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741

    theta = Cpt(SoftPositioner, limits=(-180, 180), init_pos=0, kind=NORMAL_HINTED)
    chi = Cpt(SoftPositioner, limits=(-180, 180), init_pos=0, kind=NORMAL_HINTED)
    phi = Cpt(SoftPositioner, limits=(-180, 180), init_pos=0, kind=NORMAL_HINTED)
    ttheta = Cpt(SoftPositioner, limits=(-170, 170), init_pos=0, kind=NORMAL_HINTED)

    h2 = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741
    k2 = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741
    l2 = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741

    psi = Cpt(SoftPositioner, limits=(-170, 170), init_pos=0, kind=NORMAL_HINTED)

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.set_solver("hkl_soleil", "E4CV", engine="hkl")
        self.operator.auto_assign_axes()

fourc = Fourc("", name="fourc")
print(f"{fourc=}")

fourc=Fourc(prefix='', name='fourc', settle_time=0.0, timeout=None, egu='', limits=(0, 0), source='computed', read_attrs=['h', 'h.readback', 'h.setpoint', 'k', 'k.readback', 'k.setpoint', 'l', 'l.readback', 'l.setpoint', 'theta', 'chi', 'phi', 'ttheta', 'h2', 'h2.readback', 'h2.setpoint', 'k2', 'k2.readback', 'k2.setpoint', 'l2', 'l2.readback', 'l2.setpoint', 'psi'], configuration_attrs=['geometry', 'solver', 'wavelength', 'h', 'k', 'l', 'h2', 'k2', 'l2'], concurrent=True)


In [3]:
print(f"{fourc.solver.get()=}")
print(f"{fourc.geometry.get()=}")
print(f"{fourc.wavelength.get()=}")
print(f"{fourc.solver_name=}")
print(f"{fourc.operator.solver=}")
print(f"{fourc.operator.solver.pseudo_axis_names=}")
print(f"{fourc.operator.solver.real_axis_names=}")
print(f"{fourc.operator.solver.extra_axis_names=}")
print(f"{[axis.name for axis in fourc.operator.solver.user_pseudos]=}")
print(f"{[axis.name for axis in fourc.operator.solver.user_reals]=}")
print(f"{[axis.name for axis in fourc.operator.solver.user_extras]=}")
print(f"{fourc.samples=}")
print(f"{fourc.sample=}")

fourc.solver.get()='hkl_soleil'
fourc.geometry.get()='E4CV'
fourc.wavelength.get()=1.0
fourc.solver_name='hkl_soleil'
fourc.operator.solver=HklSolver(name='hkl_soleil', version='v5.0.0.3434', geometry='E4CV', engine='hkl', mode='bissector')
fourc.operator.solver.pseudo_axis_names=['h', 'k', 'l']
fourc.operator.solver.real_axis_names=['omega', 'chi', 'phi', 'tth']
fourc.operator.solver.extra_axis_names=[]
[axis.name for axis in fourc.operator.solver.user_pseudos]=[]
[axis.name for axis in fourc.operator.solver.user_reals]=[]
[axis.name for axis in fourc.operator.solver.user_extras]=[]
fourc.samples={'cubic': Sample(name='cubic', lattice=Lattice(a=1, b=1, c=1, alpha=90.0, beta=90.0, gamma=90.0))}
fourc.sample=Sample(name='cubic', lattice=Lattice(a=1, b=1, c=1, alpha=90.0, beta=90.0, gamma=90.0))


In [4]:
print(f"{fourc.operator.axes_xref=!r}")

fourc.operator.axes_xref={'h': 'h', 'k': 'k', 'l': 'l', 'theta': 'omega', 'chi': 'chi', 'phi': 'phi', 'ttheta': 'tth'}


In [5]:
fourc.operator.remove_sample("vibranium",)
fourc.add_sample("vibranium", 2*math.pi, digits=3)
print(f"{fourc.sample=}")
print(f"{fourc.samples=}")
fourc.sample = "cubic"
print(f"{fourc.sample=}")

fourc.sample=Sample(name='vibranium', lattice=Lattice(a=6.283, b=6.283, c=6.283, alpha=90.0, beta=90.0, gamma=90.0))
fourc.samples={'cubic': Sample(name='cubic', lattice=Lattice(a=1, b=1, c=1, alpha=90.0, beta=90.0, gamma=90.0)), 'vibranium': Sample(name='vibranium', lattice=Lattice(a=6.283, b=6.283, c=6.283, alpha=90.0, beta=90.0, gamma=90.0))}
fourc.sample=Sample(name='cubic', lattice=Lattice(a=1, b=1, c=1, alpha=90.0, beta=90.0, gamma=90.0))


In [6]:
fourc.report

{'position': FourcPseudoPos(h=0, k=0, l=0, h2=0, k2=0, l2=0)}

In [7]:
fourc.summary()

data keys (* hints)
-------------------
*fourc_chi
*fourc_h
*fourc_h2
 fourc_h2_setpoint
 fourc_h_setpoint
*fourc_k
*fourc_k2
 fourc_k2_setpoint
 fourc_k_setpoint
*fourc_l
*fourc_l2
 fourc_l2_setpoint
 fourc_l_setpoint
*fourc_phi
*fourc_psi
*fourc_theta
*fourc_ttheta

read attrs
----------
h                    PseudoSingle        ('fourc_h')
h.readback           AttributeSignal     ('fourc_h')
h.setpoint           AttributeSignal     ('fourc_h_setpoint')
k                    PseudoSingle        ('fourc_k')
k.readback           AttributeSignal     ('fourc_k')
k.setpoint           AttributeSignal     ('fourc_k_setpoint')
l                    PseudoSingle        ('fourc_l')
l.readback           AttributeSignal     ('fourc_l')
l.setpoint           AttributeSignal     ('fourc_l_setpoint')
theta                SoftPositioner      ('fourc_theta')
chi                  SoftPositioner      ('fourc_chi')
phi                  SoftPositioner      ('fourc_phi')
ttheta               SoftPositioner   

In [8]:
fourc.operator.solver.geometries()

['E4CH',
 'E4CV',
 'E6C',
 'ESRF ID01 PSIC',
 'K4CV',
 'K6C',
 'PETRA3 P09 EH2',
 'PETRA3 P23 4C',
 'PETRA3 P23 6C',
 'SOLEIL MARS',
 'SOLEIL NANOSCOPIUM ROBOT',
 'SOLEIL SIRIUS KAPPA',
 'SOLEIL SIRIUS TURRET',
 'SOLEIL SIXS MED1+2',
 'SOLEIL SIXS MED2+2',
 'SOLEIL SIXS MED2+3',
 'SOLEIL SIXS MED2+3 v2',
 'TwoC',
 'ZAXIS']

In [9]:
fourc.sample.reflections.ordering = []

In [10]:
fourc.sample.reflections.add(
    hklpy2.Reflection(
        fourc.operator.solver, 
        dict(h=1, k=0, l=0), 
        dict(omega=10, chi=0, phi=0, tth=20), 
        wavelength=1.00, 
        name="r1"
    )
)
fourc.sample.reflections.add(
    hklpy2.Reflection(
        fourc.operator.solver, 
        dict(h=0, k=1, l=0), 
        dict(omega=10, chi=-90, phi=0, tth=20), 
        wavelength=1.00, 
        name="r2"
    )
)
for r in fourc.sample.reflections.ordering:
    print(f"{fourc.sample.reflections[r]}")

Reflection(name='r1', geometry='E4CV', pseudos={'h': 1, 'k': 0, 'l': 0}, angles={'omega': 10, 'chi': 0, 'phi': 0, 'tth': 20}, wavelength=1.0)
Reflection(name='r2', geometry='E4CV', pseudos={'h': 0, 'k': 1, 'l': 0}, angles={'omega': 10, 'chi': -90, 'phi': 0, 'tth': 20}, wavelength=1.0)


In [11]:
fourc.sample.reflections.swap()
print(f"{fourc.sample.reflections.ordering=}")
for r in fourc.sample.reflections.ordering:
    print(f"{fourc.sample.reflections[r]}")

fourc.sample.reflections.ordering=['r2', 'r1']
Reflection(name='r2', geometry='E4CV', pseudos={'h': 0, 'k': 1, 'l': 0}, angles={'omega': 10, 'chi': -90, 'phi': 0, 'tth': 20}, wavelength=1.0)
Reflection(name='r1', geometry='E4CV', pseudos={'h': 1, 'k': 0, 'l': 0}, angles={'omega': 10, 'chi': 0, 'phi': 0, 'tth': 20}, wavelength=1.0)
